In [ ]:
import pandas as pd
import numpy as np

try:
    df = pd.read_csv("recieved_dirty_dataset.csv", encoding='cp1252')
except UnicodeDecodeError:
    df = pd.read_csv("recieved_dirty_dataset.csv", encoding='latin1')

print("--- Info Original ---")
print(df.info())

# Estandarizamos a mayúsculas y quitamos espacios
df.columns = df.columns.str.strip().str.upper().str.replace(' ', '_')

# Renombrar columnas específicas que tenían símbolos o nombres raros
df.rename(columns={
    '+/-': 'PLUS_MINUS',
    'PLAYER_NAME': 'PLAYER_NAME', 
    'NBA_FANTASY_PTS': 'NBA_FANTASY_PTS'
}, inplace=True)

# 3. ELIMINAR DUPLICADOS
df.drop_duplicates(inplace=True)

# PLAYER_NAME: Quitar espacios al inicio y final (strip)
df['PLAYER_NAME'] = df['PLAYER_NAME'].str.strip()

# Quitar filas con "INVALID_TEAM"
df = df[df['TEAM_ABBREVIATION'] != 'INVALID_TEAM']
# b) Limpiar caracteres raros (Typos: !, ?, X) usando Regex
df['TEAM_ABBREVIATION'] = df['TEAM_ABBREVIATION'].str.replace(r'[^A-Z]', '', regex=True)

# 5. LIMPIEZA Y CONVERSIÓN DE COLUMNAS NUMÉRICAS

df['PTS'] = df['PTS'].astype(str).str.replace('pts', '', case=False)
df['PTS'] = pd.to_numeric(df['PTS'], errors='coerce')

df['FT_PCT'] = df['FT_PCT'].astype(str).str.replace(',', '.')
df['FT_PCT'] = pd.to_numeric(df['FT_PCT'], errors='coerce')

# Al usar to_numeric con errors='coerce', "Unknown" se convierte automáticamente en NaN
df['REB'] = pd.to_numeric(df['REB'], errors='coerce')

df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce')

# 6. MANEJO DE VALORES FALTANTES


# Opción A: Imputar con la media (para PTS, REB, AGE, FG_PCT)
cols_to_fill = ['PTS', 'REB', 'AGE', 'FG_PCT', 'FT_PCT']
for col in cols_to_fill:
    mean_val = df[col].mean()
    df[col].fillna(mean_val, inplace=True)


# 7. MANEJO DE OUTLIERS (Valores Extremos)

df['GP'] = df['GP'].abs()

# b) MIN (Minutos): Había un valor de 50,000.
# Un jugador no puede jugar más minutos totales de los que hay en una temporada.
# Vamos a poner un límite lógico (ej. 4000 min) y reemplazar los excesos con la media o NaN
media_min = df[df['MIN'] < 4000]['MIN'].mean()
df.loc[df['MIN'] > 4000, 'MIN'] = media_min

# 8. GUARDAR EL DATASET LIMPIO
# Importante: Guardar en UTF-8 para evitar problemas futuros
df.to_csv("clean_dataset.csv", index=False, encoding='utf-8')

print("--- Dataset Limpiado ---")
print(df.info())
print(df.head())

--- Info Original ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PLAYER NAME         589 non-null    object 
 1   TEAM_ABBREVIATION   589 non-null    object 
 2   AGE                 569 non-null    float64
 3   GP                  589 non-null    int64  
 4   W                   589 non-null    int64  
 5   L                   589 non-null    int64  
 6   MIN                 589 non-null    float64
 7   PTS                 559 non-null    object 
 8   REB                 589 non-null    int64  
 9   AST                 589 non-null    int64  
 10  STL                 589 non-null    int64  
 11  BLK                 589 non-null    int64  
 12  FG_PCT              562 non-null    float64
 13  FG3_PCT             589 non-null    float64
 14  FT_PCT              589 non-null    object 
 15  +/-                 589 non-null   